# An Efficient Machine Learning Model ?

In [1]:
import pandas as pd
import numpy as np
import nltk
import numpy as np
from sklearn.model_selection import train_test_split
import string
import re

In [2]:
from nltk import word_tokenize

In [3]:
df = pd.read_csv(r'C:\Users\Aviel\Desktop\rings.csv', error_bad_lines=False, sep=';', encoding = 'unicode_escape', engine ='python')

In [4]:
df.columns
df.rename(columns = {'Reviewed in the United States on August 18, 2020':'reviews_number',
                     '1':'rank','''Just recieved a pack of 4.Size runs way too big.The rings were dirty. The edges are not clean.One ring looked like it went through a garbage disposal. I don't know who inspects these or how they could have sent these out.Free replacement is advertised, with no hassle no return, but no instructions on how.''' : 'summary',
                     '''Garbage,Just recieved a pack of 4.Size runs way too big.The rings were dirty. The edges are not clean.One ring looked like it went through a garbage disposal. I don't know who inspects these or how they could have sent these out.Free replacement is advertised, with no hassle no return, but no instructions on how.''' : 'reviews',
                    }, inplace = True)
df = df[['reviews_number', 'rank', 'reviews']]
df = df.sample(frac=1)
df[:10]

,reviews_number,rank,reviews
724,"Reviewed in the United States on June 16, 2021",5,"Great investment.,Husband wears them everyday ..."
200,"Reviewed in the United States on July 30, 2020",3,"Nice,I like them but unfortunately they are to..."
595,"Reviewed in the United States on January 16, 2021",5,"So glaaaaad we found these!,My new hubby loves..."
426,"Reviewed in the United States on May 13, 2021",4,"Great value,Awesome ring for the money."
870,"Reviewed in the United States on August 24, 2020",5,"Great Quality for an Even Better Price!,My hus..."
1718,"Reviewed in the United States on October 25, 2020",5,"No Regrets!,Husband says he likes much better ..."
919,"Reviewed in the United States on July 6, 2020",5,"Very comfortable and a great price,I bought a ..."
2400,"Reviewed in the United States on June 6, 2021",5,"Great!,Light weight, good quality. Happy with ..."
788,"Reviewed in the United States on July 4, 2021",5,"Great,Love it and price is great. If its a bit..."
1170,"Reviewed in the United States on January 31, 2021",5,"Fits well,Fits well"


In [5]:
def to_str(text):
    return str(text).lower()
df['reviews'] = df['reviews'].apply(to_str)

In [7]:
pattern = "([\s\n:,;/?])([\"\'])([a-zאבגדהוזחטיכלמנסעפצקרשתםןץףך0-9!?;\-,.\s]+)([\"\'])" # example :"בוקר " <== : " בוקר "
pattern1 = "([\s\n:,-;/?.\)\(])([\"\'])([a-zאבגדהוזחטיכלמנסעפצקרשתםןץףך0-9]{3,})" # example : ?"בוקר " ? <==  בוקר
pattern2 = "([a-zאבגדהוזחטיכלמנסעפצקרשתםןץףך0-9\s]+)([\"\'])([a-zאבגדהוזחטיכלמנסעפצקרשתםןץףך0-9]{3,})([\"\'])" # example :בוקר " בוקר   <== "בוקר"בוקר " 
pattern3 = "([a-zאבגדהוזחטיכלמנסעפצקרשתםןץףך0-9]*)(\n)([a-zאבגדהוזחטיכלמנסעפצקרשתםןץףך0-9]*)" # example בוקר\nבוקר  <== בוקר \n בוקר

all_patterns = [pattern, pattern1, pattern2, pattern3]

def cleaning(my_txt):
    for pat in all_patterns: # for all patterns pattern1, pattern2 ....
        my_txt = str(my_txt) # convert all characters to str 
        my_txt = re.sub(pat, r'\1 \2 \3', my_txt ) # sub with space between the part of the pattern                                 
    my_txt = re.sub(' ', ' ', my_txt) # replace big space with normal space
    return(my_txt)


df['reviews'] = df['reviews'].apply(cleaning)
display(df[:5])

,reviews_number,rank,reviews
724,"Reviewed in the United States on June 16, 2021",5,"great investment.,husband wears them everyday ..."
200,"Reviewed in the United States on July 30, 2020",3,"nice,i like them but unfortunately they are to..."
595,"Reviewed in the United States on January 16, 2021",5,"so glaaaaad we found these!,my new hubby loves..."
426,"Reviewed in the United States on May 13, 2021",4,"great value,awesome ring for the money."
870,"Reviewed in the United States on August 24, 2020",5,"great quality for an even better price!,my hus..."


In [ ]:
def tokenize(text):
    return nltk.word_tokenize(text)

df['reviews'] = df['reviews'].apply(tokenize)

In [ ]:
def remove_empty_word_from(list_word):
    return [i for i in list_word if i]

df['reviews'] = df['reviews'].apply(remove_empty_word_from)
df[:5]

In [ ]:
(df[df['rank']=='Rating'].index.values)
df = df.drop(2698)

In [ ]:
def float_the(str):
    return int(str)

df['rank'] = df['rank'].apply(float_the)
df

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import pandas as pd
from nltk.probability import FreqDist
list_all_word = df['reviews'].tolist()

print('size df', len(df.index), '\n')

from functools import reduce
list_str= [reduce(lambda x,y: x+" "+y,inner_list,"") for inner_list in list_all_word]
list_str = [x for x in list_str if x]
list_str[:5]


In [ ]:
tfidfvectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1,1), max_features=500)
# convert th documents into a matrix
tfidf_wm = tfidfvectorizer.fit_transform(list_str)
tfidf_wm

In [ ]:
reviews = []
for i in range(len(list_str)):
    reviews.append(f'rewiew{i}')
tfidf_tokens = tfidfvectorizer.get_feature_names()
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),index = reviews,columns = tfidf_tokens)

pd.set_option('max_columns', None)
display(df_tfidfvect.iloc[:4])


In [59]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

X_train, X_test, y_train, y_test = train_test_split(
    df_tfidfvect, df['rank'], test_size=0.2, random_state=5)

print(X_train.shape)
print(X_test.shape)


(2158, 3495)
(540, 3495)


In [60]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
# Model Generation Using Multinomial Naive Bayes
clf = MultinomialNB().fit(X_train, y_train)
predicted_train= clf.predict(X_train)
predicted_test= clf.predict(X_test)


print("MultinomialNB Accuracy train:", accuracy_score(y_train, predicted_train))
print("MultinomialNB Accuracy test: ", accuracy_score(y_test, predicted_test))

MultinomialNB Accuracy train: 0.8229842446709916
MultinomialNB Accuracy test:  0.8111111111111111


# Is it a good result ? 
# For know that, let's see the confusion Matrix

In [61]:
confusion_matrix(y_test, predicted_test)

array([[  0,   0,   0,   0,  19],
       [  0,   0,   0,   0,   9],
       [  0,   0,   0,   0,  24],
       [  0,   0,   0,   0,  48],
       [  0,   0,   0,   2, 438]], dtype=int64)

# The model failed to categorize. The majority of the notes are at 5, which is what gives it great precision.

# What with a Random Forest ?

In [62]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV
random_forest = RandomForestClassifier()
from sklearn import metrics

In [63]:
parameters = {'n_estimators': [20,100],'criterion': ['entropy'], 'max_depth': [10,60,150]}

grid_obj = GridSearchCV(random_forest, parameters, cv=8, n_jobs = -1)
grid_obj = grid_obj.fit(X_train, y_train)
clf_forest = grid_obj.best_estimator_
print(grid_obj.best_estimator_)
# Fit the best algorithm to the data. 
clf_forest.fit(X_train, y_train)
y_pred = clf_forest.predict(X_test)
y_pred_train = clf_forest.predict(X_train)
print("Random forest train", accuracy_score(y_train, y_pred_train))
print("Random forest", accuracy_score(y_test, y_pred))


RandomForestClassifier(criterion='entropy', max_depth=60, n_estimators=20)
Random forest train 0.989805375347544
Random forest 0.8166666666666667


In [66]:
confusion_matrix(y_test, y_pred)

array([[  1,   0,   0,   0,  18],
       [  0,   0,   0,   0,   9],
       [  0,   0,   0,   0,  24],
       [  0,   0,   0,   0,  48],
       [  0,   0,   0,   0, 440]], dtype=int64)

# Here also, the model failed to categorize. The majority of the notes are at 5, which is what gives it great precision.

# Try now with Bert

In [2]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://miro.medium.com/max/700/0*ViwaI3Vvbnd-CJSQ.png")

In [ ]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [4]:
!pip install transformers 

In [79]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
import re

In [80]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [70]:
df2 = pd.read_csv(r'C:\Users\Aviel\Desktop\rings.csv', error_bad_lines=False, sep=';', encoding = 'unicode_escape', engine ='python')

df2.rename(columns = {'Reviewed in the United States on August 18, 2020':'reviews_number',
                     '1':'rank','''Just recieved a pack of 4.Size runs way too big.The rings were dirty. The edges are not clean.One ring looked like it went through a garbage disposal. I don't know who inspects these or how they could have sent these out.Free replacement is advertised, with no hassle no return, but no instructions on how.''' : 'summary',
                     '''Garbage,Just recieved a pack of 4.Size runs way too big.The rings were dirty. The edges are not clean.One ring looked like it went through a garbage disposal. I don't know who inspects these or how they could have sent these out.Free replacement is advertised, with no hassle no return, but no instructions on how.''' : 'reviews',
                    }, inplace = True)
df2 = df2[['reviews_number', 'rank', 'reviews']]

list_str = df2['reviews'].tolist()

In [71]:
reviews = list_str

In [72]:
sublist = [reviews[i] for i in y_test.index.tolist()]

In [73]:
sublist

['So comfortable!,These rings are awesome! And the customer service is out of this world!!!',
 'Ring went from 13 to 18,Got them in time looks good and fits great. In farmin industrie but sure as hell these rings dont like fuel gas or diesel. Spilled some diesle the other day and my ring went from size 13 to 18 not happy about it.',
 'Runs large,Followed directions, measured three seperate occasions and still it does not fit.',
 'Whish we found these sooner!,So glad we found these! Great color choices, fit, and style. Will be our go to brand from now on.',
 'Great Fit,Fits perfectly, great product.',
 'Great fit!,My husband likes this ring. We have had trouble finding rings in the past that fit around his knuckle and are not too big for his finger.',
 'Buy a FULL larger size.,I am normally a 6.5/6.75 in rings. I ordered size 7. These turned out to be much too small. In the packaging it suggests stretching them out before wearing but this only helped moderately. I wish i would have retu

#  Load Reviews into DataFrame and Score

In [74]:
import numpy as np
import pandas as pd

In [75]:
df2 = pd.DataFrame(np.array(sublist), columns=['review'])
df2

,review
0,"So comfortable!,These rings are awesome! And t..."
1,"Ring went from 13 to 18,Got them in time looks..."
2,"Runs large,Followed directions, measured three..."
3,"Whish we found these sooner!,So glad we found ..."
4,"Great Fit,Fits perfectly, great product."
...,...
535,"Looks nice. Good value.,Very comfortable. Fits..."
536,"Great company,Great ring. So comfortable and c..."
537,"Nice,My husband says his ring is very comforta..."
538,"Great product,he product is great and to get i..."


In [81]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [82]:
df2['predict']= df2['review'].apply(sentiment_score)

In [83]:
df2[:25]

,review,predict
0,"So comfortable!,These rings are awesome! And t...",5
1,"Ring went from 13 to 18,Got them in time looks...",2
2,"Runs large,Followed directions, measured three...",2
3,"Whish we found these sooner!,So glad we found ...",5
4,"Great Fit,Fits perfectly, great product.",5
5,"Great fit!,My husband likes this ring. We have...",5
6,"Buy a FULL larger size.,I am normally a 6.5/6....",3
7,"Seller easy to work with and great product,I o...",5
8,"Great product,Work great. True to size. Comfor...",5
9,"MUST BUY!,My boyfriend loves it!",5


In [84]:
bert_predict = df2['predict'].values

In [85]:
accuracy_score(y_test, bert_predict)

0.7555555555555555

# It maybe seems less good of the Naive bayse or Random forest 
# but let's see the confusion Matrix

In [86]:
confusion_matrix(y_test, bert_predict)

array([[  7,   9,   2,   0,   1],
       [  3,   3,   3,   0,   0],
       [  2,   5,   9,   7,   1],
       [  0,   4,  14,  13,  17],
       [  1,   1,   3,  59, 376]], dtype=int64)

# <h1> We can see here in reality that Bert's model is extremely precise and that the errors that he makes in the majority, are errors that a human could make (difference between 1 and 2 or 5 and 4) <h1>